2 BERNOULLI BANDITS FIXED POLICY

WIN STAY LOOSE SHIFT (WSLS) benchmark problem.

Controller size=2

In [1]:
import numpy as np

dimS=2
dimA=2
dimQ=2
dimO=2
p=0.8


states=np.arange(0,dimS,dtype=int)
actions=np.arange(0,dimA,dtype=int)
nodes=np.arange(0,dimQ,dtype=int)
obs=np.arange(0,dimO,dtype=int)


# p(s'|sa)
env_prob=np.zeros((dimS,dimA,dimS))

for stateprime in states:
    for action in actions:
        for state in states:
            if (stateprime == state): env_prob[stateprime,state,action]=1


# p(a|q) Here's the policty of the WSLS strategy
policy=np.zeros((dimA,dimQ))

for action in actions:
    for node in nodes:
        if (action == node): policy[action,node]=1

#f(o|s'a)
obs_prob=np.zeros((dimO,dimS,dimA))

for stateprime in states:
    for o in obs:
        for action in actions:
            if (stateprime==action): 
                obs_prob[o,stateprime,action]=p**o*(1.0-p)**(1-o)
            else:
                obs_prob[o,stateprime,action]=(1.0-p)**o*p**(1.0-o)

#g(q'|qao) Here's the memory update of the WSLS strategy
update=np.zeros((dimQ,dimA,dimQ,dimO))

for nodeprime in nodes:
    for node in nodes:
        for action in actions:
            for o in obs:
                if (o==1 and node==nodeprime): update[nodeprime,action,node,o]=1
                if (o==0 and node!=nodeprime): update[nodeprime,action,node,o]=1
                
#r(sa) Average reward
rew=np.zeros((dimS,dimA))

rew[0,0]=p
rew[0,1]=1-p
rew[1,0]=1-p
rew[1,1]=p


Since this is a benchmark problem the policy and the memory update have been fixed. The Bellman quaratic constraint boils down to be a linear system. The solution of this system is the value. 

In [2]:
#b(sq)
init_node=0
gamma=0.5
b=np.matmul(rew,policy)

#Bellman equation linear constraint
T=np.zeros((dimQ,dimS,dimQ,dimS))
Id=np.zeros((dimQ,dimS,dimQ,dimS))

#T(sq|s'q')=sum_a sum_y (p(s'|sa)*f(o|sa)*g(q'|qao)*policy(a|q))
for node in nodes:
    for state in states:
        for nodeprime in nodes:
            for stateprime in states:
                for o in obs:
                    for action in actions:
                        T[node,state,nodeprime,stateprime]+=update[nodeprime,action,node,o]*policy[action,node]*env_prob[stateprime,state,action]*obs_prob[o,stateprime,action]

#Id=delta(ss',qq')
for nodeprime in nodes:
    for stateprime in states:
        for node in nodes:
            for state in states:
                if (state==stateprime and node==nodeprime): Id[node,state,nodeprime,stateprime]=1

#delta(ss',qq')- gamma*[sum_a sum_y (p(s'|sa)*f(o|sa)*g(q'|qao)*policy(a|q))]
Btens=Id-gamma*T

#Solve the linear system to get the single value solution
v=np.linalg.tensorsolve(Btens,b)
print("Value:",v)
print("=============")

#Calculation of the objective function with fixed inotial node
print("Objective function:", 0.5*np.sum(v[init_node,:]))

#Def normalized x(q',a,q,o)
x=np.zeros((dimQ,dimA,dimQ,dimO))
for nodeprime in nodes:
    for action in actions:
        for node in nodes:
            for o in obs:
                x[nodeprime,action,node,o]=update[nodeprime,action,node,o]*policy[action,node]



Value: [[1.48 0.88]
 [0.88 1.48]]
Objective function: 1.18
0.0 1.0


In the following section the same calculations are carried out. In this case rank two arrays have been used to evalute the Bellman equation. 


The "Bellman matrix" (Bmat) is block diagonal, which makes sense because the transition probability from one stae to the other is zero.

In [3]:
import numpy as np
from f2py_jit import jit
f90=jit("SA_QCLP_ut_02_cor1.f90",flags="-O3 -fcheck=bounds")

#initial value for the x(q'aqo)
x_init=x
#initial value for the y(qs)
y_init=v
#Initial value of objective function
val_init=0.5*np.sum(y_init[init_node,:])

print("Initial mean value   >>>>>>>>>>")
print(val_init)
print("Initial policy       >>>>>>>>>>")
print(policy)

#Initial parameters
T0=10
T_end=1e-6
Temp_factor=0.4
factor=0.3
nstep=500
pen_step=1000
N=1
stdev=1.0

size_gauss_x=round(dimO*nstep*(1.0-factor))
size_cauchy_x=round(dimO*nstep*(factor))

size_gauss_y=round(dimS*nstep*(1.0-factor))
size_cauchy_y=round(dimS*nstep*(factor))

x=np.zeros((dimQ,dimA,dimQ,pen_step*dimO*nstep*round(1+np.log(T_end/T0)/np.log(Temp_factor))))
y=np.zeros((dimQ,pen_step*dimS*nstep*round(1+np.log(T_end/T0)/np.log(Temp_factor))))


for p in range(N):

    sigma=2.0
    mu=2.0

    for j in range(round(pen_step*(1+np.log(T_end/T0)/np.log(Temp_factor)))):

        #x(q'aqo) sample size and generation

        #Generarion of gauss sample. In this way the point generated is independent from o
        x_gauss=np.random.normal(0.0,stdev,size=(dimQ,dimA,dimQ,size_gauss_x))
        x_cauchy=np.random.standard_cauchy(size=(dimQ,dimA,dimQ,size_cauchy_x))
        a=np.append(x_cauchy,x_gauss,axis=3)
        x[:,:,:,j*dimO*nstep:(j+1)*dimO*nstep]=a
        #Generating the effective gauss_sample                 

        #y(qs) sample size and generation

        y_cauchy=np.random.standard_cauchy(size=(dimQ,size_cauchy_y))
        y_gauss=np.random.normal(0.0,stdev,size=(dimQ,size_gauss_y))
        b=np.append(y_cauchy,y_gauss,axis=1)
        y[:,j*dimO*nstep:(j+1)*dimO*nstep]=b
        
    print("The samples have been generated")
    
    #Simulated Annealing 
    Res=f90.qclp.drive(T0,T_end,Temp_factor,nstep,pen_step,x_init,y_init,val_init,rew,env_prob,obs_prob,sigma,mu,x,y,gamma)

 
x_min=Res[0]
y_min=Res[1]
print("value                >>>>>>>>>>>")
print(y_min)
print("mean value           >>>>>>>>>>>")
print(0.5*np.sum(y_min[init_node,:]))
print("objective function   >>>>>>>>>>>")
print(Res[2])


Initial mean value   >>>>>>>>>>
1.18
Initial policy       >>>>>>>>>>
[[1. 0.]
 [0. 1.]]
value                >>>>>>>>>>>
[[1.47132619 0.89380637]
 [0.80775609 1.50778072]]
mean value           >>>>>>>>>>>
1.182566277662852
objective function   >>>>>>>>>>>
-1.1805391503955724


In [4]:
print(y_min)
Pol0=np.zeros((dimA,dimQ))
Pol1=np.zeros((dimA,dimQ))

for action in actions:
    for node in nodes:
        for nodeprime in nodes:
            Pol0[action,node]=np.sum(x_min[:,action,node,0])

for action in actions:
    for node in nodes:
        Pol1[action,node]=np.sum(x_min[:,action,node,1])


print(Pol0)
print(Pol1)

normalization=np.zeros((dimQ,dimO))

for node in nodes:
    for o in obs:
        for nodeprime in nodes:
            for action in actions:
                normalization[node,o]=np.sum(x_min[:,:,node,o])

print("norm",normalization)

[[1.47132619 0.89380637]
 [0.80775609 1.50778072]]
[[9.99980402e-01 2.55263169e-05]
 [1.95981791e-05 9.99974474e-01]]
[[9.99992705e-01 3.27820311e-06]
 [7.29546779e-06 9.99996722e-01]]
norm [[1. 1.]
 [1. 1.]]


In [1]:
import numpy as np

dimS=2
dimA=2
dimQ=3
dimO=2
p=0.8


states=np.arange(0,dimS,dtype=int)
actions=np.arange(0,dimA,dtype=int)
nodes=np.arange(0,dimQ,dtype=int)
obs=np.arange(0,dimO,dtype=int)


# p(s'|sa)
env_prob=np.zeros((dimS,dimA,dimS))

for stateprime in states:
    for action in actions:
        for state in states:
            if (stateprime == state): env_prob[stateprime,state,action]=1


# p(a|q) Here's the policty of the WSLS strategy
policy=np.zeros((dimA,dimQ))

for action in actions:
    for node in nodes:
        if (action == node):
            policy[action,node]=1
        elif (node==2):
            policy[action,node]=0.5
        else:
            policy[action,node]=0

#f(o|s'a)
obs_prob=np.zeros((dimO,dimS,dimA))

for stateprime in states:
    for o in obs:
        for action in actions:
            if (stateprime==action): 
                obs_prob[o,stateprime,action]=p**o*(1.0-p)**(1-o)
            else:
                obs_prob[o,stateprime,action]=(1.0-p)**o*p**(1.0-o)

#g(q'|qao) Here's the memory update of the WSLS strategy
update=np.zeros((dimQ,dimA,dimQ,dimO))

for nodeprime in nodes:
    for action in actions:
        for node in nodes:
            for o in obs:
                # if we win in the node corresponding to action, we stay
                if nodeprime == node and node == action and o == 1:
                    update[nodeprime,action,node,o] = 1
                # if we lose in the node corresponding to action, we shift to the intermediate node
                elif nodeprime == 2 and node == action and o == 0:
                    update[nodeprime,action,node,o] = 1
                # if we win in the intermediate node, we shift to node corresponding to action taken
                elif nodeprime == action and node == 2 and o == 1:
                    update[nodeprime,action,node,o] = 1 # P(a | 2) = 0.5
                # if we lose in the intermediate node, we shift to node corresponding to the other action
                elif nodeprime!= action and node == 2 and nodeprime != 2 and o == 0:
                    update[nodeprime,action,node,o] = 1 # P(a | 2) = 0.5
                # the rest is 0
                else:
                    update[nodeprime,action,node,o] = 0
                
#r(sa) Average reward
rew=np.zeros((dimS,dimA))

rew[0,0]=p
rew[0,1]=1-p
rew[1,0]=1-p
rew[1,1]=p


In [9]:
#b(sq)
init_node=0
gamma=0.5
b=np.zeros((dimQ,dimS))

for node in nodes:
    for state in states:
        for action in actions:
            b[node,state]+=rew[state,action]*policy[action,node]


#Bellman equation linear constraint
T=np.zeros((dimQ,dimS,dimQ,dimS))
Id=np.zeros((dimQ,dimS,dimQ,dimS))

#T(qs|q's')=sum_a sum_y (p(s'|sa)*f(o|sa)*g(q'|qao)*policy(a|q))
for node in nodes:
    for state in states:
        for nodeprime in nodes:
            for stateprime in states:
                for o in obs:
                    for action in actions:
                        T[node,state,nodeprime,stateprime]+=update[nodeprime,action,node,o]*policy[action,node]*env_prob[stateprime,state,action]*obs_prob[o,stateprime,action]

#Id=delta(ss',qq')
for nodeprime in nodes:
    for stateprime in states:
        for node in nodes:
            for state in states:
                if (state==stateprime and node==nodeprime): Id[node,state,nodeprime,stateprime]=1

#delta(ss',qq')- gamma*[sum_a sum_y (p(s'|sa)*f(o|sa)*g(q'|qao)*policy(a|q))]
Btens=Id-gamma*T

#Solve the linear system to get the single value solution
v=np.linalg.tensorsolve(Btens,b)
print("Value:",v)
print("=============")

#Calculation of the objective function with fixed inotial node
print("Objective function:", 0.5*np.sum(v[init_node,:]))

#Def normalized x(q',a,q,o)
x=np.zeros((dimQ,dimA,dimQ,dimO))
for nodeprime in nodes:
    for action in actions:
        for node in nodes:
            for o in obs:
                x[nodeprime,action,node,o]=update[nodeprime,action,node,o]*policy[action,node]

Value: [[1.53125 0.75   ]
 [0.75    1.53125]
 [1.1875  1.1875 ]]
Objective function: 1.140625


In [20]:
import numpy as np
from f2py_jit import jit
f90=jit("SA_QCLP_ut_02_cor1.f90",flags="-O3 -fcheck=bounds")

#initial value for the x(q'aqo)
x_init=x
#initial value for the y(qs)
y_init=v
#Initial value of objective function
val_init=0.5*np.sum(y_init[init_node,:])

print("Initial mean value   >>>>>>>>>>")
print(val_init)
print("Initial policy       >>>>>>>>>>")
print(policy)

#Initial parameters
T0=10
T_end=1e-6
Temp_factor=0.3
factor=0.4
nstep=50
pen_step=350
N=10
stdev=1.0

xmin_temp=x_init
ymin_temp=y_init
valmin_temp=-val_init

size_gauss_x=round(dimO*nstep*(1.0-factor))
size_cauchy_x=round(dimO*nstep*(factor))

size_gauss_y=round(dimS*nstep*(1.0-factor))
size_cauchy_y=round(dimS*nstep*(factor))

rndx=np.zeros((dimQ,dimA,dimQ,pen_step*dimO*nstep*round(1+np.log(T_end/T0)/np.log(Temp_factor))))
rndy=np.zeros((dimQ,pen_step*dimS*nstep*round(1+np.log(T_end/T0)/np.log(Temp_factor))))


for p in range(N):

    sigma=2.0
    mu=2.0

    for j in range(pen_step*round(1+np.log(T_end/T0)/np.log(Temp_factor))):

        #x(q'aqo) sample size and generation

        #Generarion of gauss sample. In this way the point generated is independent from o
        x_gauss=np.random.normal(0.0,stdev,size=(dimQ,dimA,dimQ,size_gauss_x))
        x_cauchy=np.random.standard_cauchy(size=(dimQ,dimA,dimQ,size_cauchy_x))
        a=np.append(x_cauchy,x_gauss,axis=3)
        rndx[:,:,:,j*dimO*nstep:(j+1)*dimO*nstep]=a
        #Generating the effective gauss_sample                 

        #y(qs) sample size and generation

        y_cauchy=np.random.standard_cauchy(size=(dimQ,size_cauchy_y))
        y_gauss=np.random.normal(0.0,stdev,size=(dimQ,size_gauss_y))
        b=np.append(y_cauchy,y_gauss,axis=1)
        rndy[:,j*dimO*nstep:(j+1)*dimO*nstep]=b
    
    #Simulated Annealing 
    Res=f90.qclp.drive(T0,T_end,Temp_factor,nstep,pen_step,x_init,y_init,val_init,rew,env_prob,obs_prob,sigma,mu,rndx,rndy,gamma)
    
    if (Res[2] < valmin_temp):
        x_init=Res[0]
        x_min=Res[0]
        y_init=Res[1]
        y_min=Res[1]
        val_init=Res[2]
        valmin_temp=Res[2]

    print(p,"Gone. Objective function:", Res[2])

print("value                >>>>>>>>>>>")
print(y_min)
print("mean value           >>>>>>>>>>>")
print(0.5*np.sum(y_min[init_node,:]))
print("objective function   >>>>>>>>>>>")
print(valmin_temp)


Initial mean value   >>>>>>>>>>
1.140625
Initial policy       >>>>>>>>>>
[[1.  0.  0.5]
 [0.  1.  0.5]]
0 Gone. Objective function: -1.1451855741232482
1 Gone. Objective function: -1.1408236952484039
2 Gone. Objective function: -1.1260700230466523
3 Gone. Objective function: -1.1153442284532509
4 Gone. Objective function: -1.1481731680949216
5 Gone. Objective function: -1.1706387141994108
6 Gone. Objective function: -1.1761967124699142
7 Gone. Objective function: -1.1774795978315056
8 Gone. Objective function: -1.1671268850938872
9 Gone. Objective function: -1.1664711704211361
value                >>>>>>>>>>>
[[1.46705579 0.88825209]
 [0.80151134 1.49952763]
 [1.087602   1.1749406 ]]
mean value           >>>>>>>>>>>
1.1776539408945634
objective function   >>>>>>>>>>>
-1.1774795978315056


In [21]:
print(y_min)
Pol0=np.zeros((dimA,dimQ))
Pol1=np.zeros((dimA,dimQ))

for action in actions:
    for node in nodes:
        for nodeprime in nodes:
            Pol0[action,node]=np.sum(x_min[:,action,node,0])

for action in actions:
    for node in nodes:
        Pol1[action,node]=np.sum(x_min[:,action,node,1])


print(Pol0)
print(Pol1)

normalization=np.zeros((dimQ,dimO))

for node in nodes:
    for o in obs:
        for nodeprime in nodes:
            for action in actions:
                normalization[node,o]+=x_min[nodeprime,action,node,o]

#for node in nodes:
    #for o in obs:
        #for nodeprime in nodes:
            #for action in actions:
                #if (x_min[nodeprime,action,node,o]/Pol1[action,node] > 1e-4): 
                    #print("Node:",node,">>>","Action:",action,">>>","Obs:",o,">>>","Nodeprime:",nodeprime,"::::::",x_min[nodeprime,action,node,o]/Pol0[action,node])

print("norm",normalization)



[[1.46705579 0.88825209]
 [0.80151134 1.49952763]
 [1.087602   1.1749406 ]]
[[9.99993712e-01 5.80892873e-04 5.25792177e-01]
 [6.28808598e-06 9.99419107e-01 4.74207823e-01]]
[[9.99993950e-01 5.81237287e-04 5.25792124e-01]
 [6.04960896e-06 9.99418763e-01 4.74207876e-01]]
norm [[1. 1.]
 [1. 1.]
 [1. 1.]]


In [22]:
No=np.zeros((dimA,dimQ,dimO))

for nodeprime in nodes:
    for action in actions:
        for node in nodes:
            for o in obs:
                No[action,node,o]+=x_min[nodeprime,action,node,o]/Pol0[action,node]


for action in actions:
    for node in nodes:
        for o in obs:
            print(action,node,o,No[action,node,o])

0 0 0 0.9999999999999999
0 0 1 1.000000238478518
0 1 0 0.9999999999999999
0 1 1 1.0005929040651158
0 2 0 1.0
0 2 1 0.9999998985889069
1 0 0 0.9999999999999999
1 0 1 0.9620747840910093
1 1 0 1.0
1 1 1 0.9999996553860704
1 2 0 1.0
1 2 1 1.0000001124425981
